# **Elements of Machine Learning 2024** <font size=4 color='gray'>Alan Reyes-Figueroa</font>
#### <font color='gray'>Kernel Density Estimator (KDE)</font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def relu(x):
    return x*(x>0)

In [ ]:
def basef(x, x0, h=1.0, kernel='tophat'):
    if (kernel == 'tophat'):
        f = (np.abs(x-x0) < h).astype(type(x[0]))
    elif (kernel == 'gaussian'):
        f = np.exp(-(x-x0)**2/ (2.*h**2))
    elif (kernel == 'laplacian'):
        f = np.exp(-np.abs(x-x0)/h)
    elif (kernel == 'linear'):
        f = relu(1. - np.abs((x-x0)/h))
    elif (kernel == 'cosine'):
        f = (np.abs(x-x0)<h)*np.cos((x-x0)*np.pi/(2.*h))
    elif (kernel == 'epanechnikov'):
        f = relu(1. - (x-x0)**2/(h**2))
    else: 
        f = 0.*x
    return f

In [ ]:
x = np.linspace(-10, 10., 201)
kernels = ['tophat', 'linear', 'gaussian', 'laplacian', 'cosine', 'epanechnikov']
x0 = 0.
h = 5.

plt.figure(figsize=(12,6))
for i in range(0, 6):
    plt.subplot(2,3,i+1)
    y = basef(x, x0, h=h, kernel=kernels[i])
    plt.plot(x, y)
    plt.title(kernels[i])
plt.show()

## Random data

In [ ]:
data = np.random.normal(0., 1., (100,)) + 0.2*np.random.normal(7., 5., (100,))
data.shape

In [ ]:
np.round(data, 3)[:25]

In [ ]:
def density(x, n=100, h=1., kernel='tophat'):
    std = x.std()
    minn = x.min() - 2.*std
    maxx = x.max() + 2.*std
    t = np.linspace(minn, maxx, n+1)
    dens = np.zeros(t.shape)
    for i in range(0, x.shape[0]):
        dens = dens + basef(t, x0=x[i], h=h, kernel=kernel)
    dens = dens/x.shape[0]
    return t, dens

In [ ]:
n = data.shape[0]
bns = int(np.ceil(np.log2(n))) + 1
print(bns)
plt.figure(figsize=(4,4))
plt.hist(data, bins=bns)
plt.show()

In [ ]:
h = [0.5, 0.7, 0.9]

t1, d1 = density(data, n=200, h=h[0], kernel='cosine')
t2, d2 = density(data, n=200, h=h[1], kernel='cosine')
t3, d3 = density(data, n=200, h=h[2], kernel='cosine')

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.plot(t1, d1)
plt.title('h = {}'.format(h[0]))
plt.subplot(1,3,2)
plt.plot(t2, d2)
plt.title('h = {}'.format(h[1]))
plt.subplot(1,3,3)
plt.plot(t3, d3)
plt.title('h = {}'.format(h[2]))
plt.show

In [ ]:
t, d = density(data, n=200, h=0.1, kernel='tophat')

In [ ]:
kernels = ['tophat', 'linear', 'gaussian', 'laplacian', 'cosine', 'epanechnikov']
n = 100
h = 1.

plt.figure(figsize=(8,6))
for i in range(0, 6):
    plt.subplot(2,3,i+1)
    t, d = density(data, n=n, h=h, kernel=kernels[i])
    plt.plot(t, d)
    plt.title(kernels[i])
plt.show()

## Scikit-learn

In [ ]:
from sklearn.neighbors import KernelDensity
from scipy.stats import norm

In [ ]:
# Plot a 1D density example
N = 500
np.random.seed(1)
X = np.concatenate((np.random.normal(0, 1, int(0.3 * N)),
                    np.random.normal(5, 1, int(0.7 * N))))[:, np.newaxis]

In [ ]:
X.shape

In [ ]:
X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]

In [ ]:
true_dens = (0.3 * norm(0, 1).pdf(X_plot[:, 0])
             + 0.7 * norm(5, 1).pdf(X_plot[:, 0]))

In [ ]:
fig, ax = plt.subplots()
ax.fill(X_plot[:, 0], true_dens, fc='black', alpha=0.2,
        label='input distribution')
colors = ['navy', 'cornflowerblue', 'darkorange']
kernels = ['gaussian', 'tophat', 'epanechnikov']
lw = 2

for color, kernel in zip(colors, kernels):
    kde = KernelDensity(kernel=kernel, bandwidth=0.5).fit(X)
    log_dens = kde.score_samples(X_plot)
    ax.plot(X_plot[:, 0], np.exp(log_dens), color=color, lw=lw, linestyle='-', label="kernel = '{0}'".format(kernel))

ax.text(6, 0.38, "N={0} points".format(N))

ax.legend(loc='upper left')
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), '+k')

ax.set_xlim(-4, 9)
ax.set_ylim(-0.02, 0.4)
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure()
sns.displot(X, bins=51, kde=True)
plt.show()

## Aproximación de funciones

In [ ]:
x = np.linspace(0, 10, 101)

In [ ]:
centers = np.linspace(0.5, 9.5, 16)
centers

In [ ]:
#f = 0.75*np.cos(1.7*x) + 1.3*np.cos(1.1*x) + 0.5*np.sin(0.1*x)

f = 1.*np.cos(2.7*x) + 1.7*np.cos(1.35*x) + 0.5*np.sin(2.1*x)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(x, f)
plt.show()

In [ ]:
n = x.shape[0]
d = centers.shape[0]
print(n, d)

In [ ]:
X = np.zeros((d, n))
sigma = 1.

for i in range(0, d):
    X[i,:] = basef(x, centers[i], h=sigma, kernel='linear')

In [ ]:
plt.figure(figsize=(5,5))
for i in range(0, d):
    plt.plot(x, X[i])
plt.show()

In [ ]:
y = X @ f
a = np.linalg.inv(X@(X.T)) @ y

In [ ]:
a

In [ ]:
f.shape

In [ ]:
plt.figure(figsize=(5,5))
for i in range(0, d):
    plt.plot(x, a[i]*X[i])
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(x, f)
plt.plot(x, X.T@a)
plt.show()